## 5.4 池化层

> **连续拍摄同一物体在不同图像中也有可能出现像素位置的偏移,从而导致卷积输出在不同的位置,池化层就是为了【缓解卷积层对位置的过度敏感性】**

### 5.4.1 二维最大池化层和平均池化层

> 1. **池化层直接对池化窗口内元素取最大或平均值**
> 2. **池化窗口形状为$p \times q$叫做$p \times q$池化**

In [1]:
import torch
from torch import nn

In [2]:
def pool2d(X, pool_size, mode='max'):
    X = X.float()
    p_h, p_w = pool_size
    Y = torch.zeros(X.shape[0] - p_h + 1, X.shape[1] - p_w + 1)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i:i+p_h, j:j+p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i: i+p_h, j:j+p_w].mean()
    return Y

In [3]:
X = torch.tensor([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
pool2d(X, (2, 2))

tensor([[4., 5.],
        [7., 8.]])

In [4]:
pool2d(X, (2, 2), 'avg')

tensor([[2., 3.],
        [5., 6.]])

### 5.4.2 填充和步幅

> **`nn.MaxPool2d`默认步幅和池化窗口相同**

In [5]:
# 形状为[1, 1, 4, 4], 前两个分别为输出和输入通道
X = torch.arange(16, dtype=torch.float).view(1, 1, 4, 4)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

In [6]:
pool2d = nn.MaxPool2d(3)
pool2d(X)

tensor([[[[10.]]]])

In [7]:
# 指定填充和步幅
# 输出长宽为2 = (4-3+2*1)/3 + 1
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X).shape

torch.Size([1, 1, 2, 2])

In [8]:
# 输出 3 = (4-2+2*1)/2 + 1
# 输出 2 = (4-4+2*2)/3 + 1
# 不够整除取下限
pool2d = nn.MaxPool2d((2, 4), padding=(1, 2), stride=(2, 3))
pool2d(X).shape

torch.Size([1, 1, 3, 2])

### 5.4.3 多通道

> **对输入数据的每个通道进行池化,所以输入和输出通道数相等**

In [9]:
# 在给定维度上对输入的张量序列seq 进行连接操作。
X = torch.cat((X, X+1), dim=1)
print(X)
print(X.shape)

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])
torch.Size([1, 2, 4, 4])
